# **Import Libraries**

In [1]:
import joblib
import numpy as np
import pandas as pd
import tensorflow as tf
from TF_Object import L2NormalizeLayer

# **Prepare Dataset**

In [2]:
# Load all dataset in Pandas DataFrame
tourism_df = pd.read_csv("../Dataset/new_tourism_with_id_links_3.csv")

In [3]:
tourism_df.head(3)

,Place_Id,Place_Name,Category,Description,City,Price,Rating,Time_Minutes,Coordinate,Lat,Long,Img_Path_0,Img_Path_1,Img_Path_2
0,1,Monumen Nasional,"Budaya, Situs Sejarah",Monumen Nasional atau yang populer disingkat d...,Jakarta,20000,"4,6",15.0,"{'lat': -6.1753924, 'lng': 106.8271528}",-6.175392,106.827153,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...,images_output\Monumen Nasional\Monumen Nasiona...
1,2,Kota Tua,Budaya,"Kota tua di Jakarta, yang juga bernama Kota Tu...",Jakarta,0,"4,6",90.0,"{'lat': -6.137644799999999, 'lng': 106.8171245}",-6.137645,106.817125,images_output\Kota Tua\Kota Tua_0.jpg,images_output\Kota Tua\Kota Tua_1.jpg,NaN
2,3,Dunia Fantasi,"Taman Hiburan, Bahari",Dunia Fantasi atau disebut juga Dufan adalah t...,Jakarta,270000,"4,6",360.0,"{'lat': -6.125312399999999, 'lng': 106.8335377}",-6.125312,106.833538,images_output\Dunia Fantasi\Dunia Fantasi_0.jpg,images_output\Dunia Fantasi\Dunia Fantasi_1.png,images_output\Dunia Fantasi\Dunia Fantasi_2.jpg


# **Initiate Important Instance**

In [ ]:
# Load Vector Data
user_data_vecs = np.load('../Vector/user_vector.npy')        # Example of transformation result from user DB
tourism_data_vecs = np.load('../Vector/tourism_vector.npy')  # Example of transformation result from tourism DB

# Load Scaler
user_scaler = joblib.load('../Scaler/user_scaler.gz')
tourism_scaler = joblib.load('../Scaler/tourism_scaler.gz')
target_scaler = joblib.load('../Scaler/target_scaler.gz')

# Load NN Model
cb_model = tf.keras.models.load_model('Model/content_based.h5', custom_objects={
    'L2NormalizeLayer': L2NormalizeLayer
})

# **Content-Based Recommendation**

In [6]:
# Make a prediction using content-based model
def get_recommendations(cb_model: tf.keras.Model, user_id: int) -> tuple[np.ndarray, np.ndarray]:
  # Prepare User Data
  current_user_data = user_data_vecs[user_id]
  current_user_vecs = np.tile(current_user_data, (tourism_data_vecs.shape[0], 1))
  scaled_current_user_vecs = user_scaler.transform(current_user_vecs)

  # Prepare Tourism Data
  scaled_tourism_vecs = tourism_scaler.transform(tourism_data_vecs)

  # Make prediction
  y_pred = cb_model.predict([scaled_current_user_vecs, scaled_tourism_vecs])
  y_pred = target_scaler.inverse_transform(y_pred)

  # Sort the prediction results
  sorted_index = np.argsort(-y_pred, axis=0).reshape(-1)
  sorted_y_pred = y_pred[sorted_index].reshape(-1)

  return sorted_index, sorted_y_pred

# Display recommendations
def print_recommendations(user_id: int, city: str, num_recom: int, sorted_index: np.ndarray, sorted_y_pred: np.ndarray) -> None:
  # Display user data
  columns = ['Bahari', 'Budaya', 'Cagar Alam', 'Kebun Binatang', 'Pusat Perbelanjaan',
             'Situs Sejarah', 'Taman Hiburan', 'Teater & Pertunjukan', 'Tempat Ibadah']
  current_user_data = user_data_vecs[user_id]
  current_user_data_df = pd.DataFrame(data=current_user_data.reshape(1, 9), columns=columns)
  print(f'Getting recommendation for user_id = {user_id}, with following data')
  display(current_user_data_df)
  
  # Display recommendation tourist spot
  print(f'Diplay {num_recom} tourist spots recommendation in {city}')
  result = tourism_df.loc[sorted_index, ['Place_Name', 'Category', 'City']]
  result['Rating_Prediction'] = sorted_y_pred
  display(result.loc[result['City'] == city, :].head(num_recom))

In [7]:
# Print 20 recommendations for user_id = 5 in Jakarta
user_id = 5
city = 'Jakarta'
num_recom = 20

sorted_index, sorted_y_pred = get_recommendations(cb_model, user_id)
print_recommendations(user_id, city, num_recom, sorted_index, sorted_y_pred)

14/14 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Getting recommendation for user_id = 5, with following data


,Bahari,Budaya,Cagar Alam,Kebun Binatang,Pusat Perbelanjaan,Situs Sejarah,Taman Hiburan,Teater & Pertunjukan,Tempat Ibadah
0,2.0,3.9,3.1,0.0,0.0,4.0,3.1,2.0,0.0


Diplay 20 tourist spots recommendation in Jakarta


,Place_Name,Category,City,Rating_Prediction
68,Freedom Library,"Budaya, Situs Sejarah",Jakarta,4.237989
48,Galeri Indonesia Kaya,"Budaya, Situs Sejarah",Jakarta,4.233280
73,Museum Tengah Kebun,"Budaya, Situs Sejarah",Jakarta,4.225269
64,Museum Basoeki Abdullah,"Budaya, Situs Sejarah",Jakarta,4.217810
58,Museum Sumpah Pemuda,"Budaya, Situs Sejarah",Jakarta,4.217377
20,Museum Wayang,"Budaya, Situs Sejarah",Jakarta,4.216676
62,Museum Kebangkitan Nasional,"Budaya, Situs Sejarah",Jakarta,4.213937
63,Museum Sasmita Loka Ahmad Yani,"Budaya, Situs Sejarah",Jakarta,4.213937
65,Museum Layang-layang,"Budaya, Situs Sejarah",Jakarta,4.208267
52,Rumah Sipitung,"Budaya, Situs Sejarah",Jakarta,4.207789
